# University Of Pennsylvania

## Campus Jobs Retriever and Organizer

#### Why the need arose? 


I am attending Univeristy of Pennsylvania in Fall 2021 for the program Masters in Data Science. 
I wanted to look at all the job opportunities I could pursue with my program at one glance and also sort or filter them on criterions such as Job Description, hourly pay, number of hours per week or weekly pay.
This script was created to provide a solution to this need.

The script neatly condenses all the listed jobs on University of Pennsylvania's official jobs portal into one Excel sheet sorted in descending order of Weekly Pay.

## Import Libraries

In [14]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import re
import imaplib
import email

from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import timezone
import os
import ssl
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from selenium.webdriver.common.action_chains import ActionChains


## Open Selenium Controlled Chrome Browser

In [15]:
def open_browser():
    opts = Options()
    opts.headless = False
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts)
    return driver

In [16]:
driver=open_browser()

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Shrivats Agrawal\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


## Open UPenn Jobs Portal

In [17]:
driver.get("https://www.sfs.upenn.edu/seo/job_search/job_search.php")

In [18]:
for element in driver.find_elements_by_tag_name("input"):
    if("Search >>" in element.get_attribute("value")):
        print(element.get_attribute("value"))
        element.click()
        break

Search >>


## Fetching All Job IDS

In [19]:
job_ids=list()
soup=BeautifulSoup(driver.page_source,"html5lib")
for tr in soup.find_all("table",{"class":"search"})[0].find_all("tr"):
    try:
        job_anchor=tr.find("a")
        job_id=job_anchor['href']
        #print(job_id)
        job_ids.append("https://www.sfs.upenn.edu/seo/job_search/job_search.php"+job_id)
    except:
        pass
        #print(tr,"\n_______________________________________")
driver.quit()

## Fetching Details for Each Job Id

In [20]:
jobs_list=list()
for i in range(0,len(job_ids)):
    print(i)
    job=job_ids[i]
    data=requests.get(job).content
    soup=BeautifulSoup(data,"html5lib")
    job_title="-"
    job_id="-"
    posted_on="-"
    try:
        job_title=soup.find("div",{"class":"section"}).find_all("div")[0].text.strip()
    except:
        pass
    for div in soup.find("div",{"class":"section"}).find_all("div"):
        if("Job" in div.text):
            job_id=str(div.text.strip()).replace("Job","").strip()
        if("Posted on" in div.text):
            posted_on=div.text.strip().replace("Posted on","").strip()

    summary="-"
    details="-"
    contact="-"
    for div in soup.find_all("div",{"class":"section"}):
        if("Summary" in div.text):
            summary=div.text.replace("Summary","").strip().replace("\t","")
        if("Details" in div.text):
            details=div.text.replace("Details","").strip().replace("\t","")
        if("Contact Information" in div.text):
            contact=div.text.replace("Contact Information","").strip().replace("\t","")
    try:
        pay_per_hour=re.findall(".*?([$].*?) .*?",details)[0]
    except:
        pay_per_hour="-"
    try:
        hours_per_week=re.findall(".*?([0-9]{1,2}) hours per week",details)[0]
    except:
        hours_per_week="-"
    try:
        pay_per_week="$"+str(float(pay_per_hour.replace("$",""))*float(hours_per_week))
    except:
        pay_per_week="-"
    job_dict={"JOB_ID":job_id,"Title":job_title,"Posted On":posted_on,"Summary":summary,"Details":details,
              "Contact Information":contact,"Link":job,"PayPerHour":pay_per_hour,"HoursPerWeek":hours_per_week,
             "WeeklyPay":pay_per_week}
    jobs_list.append(job_dict)
    for key in job_dict.keys():
        print(key,":",job_dict[key])
    print("_____________________________________________________________")

0
JOB_ID : #25176
Title : RESEARCH ASSISTANT
Posted On : 01/08/21
Summary : The Penn Collaborative for CBT and Implementation Science within the Department of Psychiatry is seeking a research assistant to support research for the training, dissemination, and implementation of cognitive behavioral therapy (CBT) and other treatments in mental health agencies, primarily in Philadelphia. The main duties of the position will include data collection and entry as well as other research responsibilities. Work is currently virtual due to COVID19, but may return to in-person when the public health crisis allows. Work hours are flexible. Meetings with the supervisor (Dr. Amber Calloway) will occur during weekday working hours.Please email Amber Calloway with resumes: amber.calloway@pennmedicine.upenn.edu
Details : Work-Study Only1 position available for academic year 
$10.71 per hour
 / 10 hours per week
 
Requires advanced skills

Requires advanced skills. Prior work experience and/or completion

JOB_ID : #25104
Title : Student Research Assistant
Posted On : 11/18/20
Summary : The primary function and purpose of the Research Assistant position is to support the research team's study goals abiding by the IRB and HIPAA privacy rules. This position will support two graduate Nursing students, recipients of an internal Provost award and mentored by Dr. Nancy Hodgson.
Details : Both Work-Study and Non Work-Study1 position available for academic year 
$12.00 per hour
 / 8 hours per week
 
Requires advanced skills

Requires advanced skills. Prior work experience and/or completion of related course work. Responsibilities may include data analysis, independent supervision or coordination of programs and/or projects involving complex equipment or analysis of data. Often requires the ability to communicate skills or expertise to others. Can include regular supervision of other student employees and limited on site supervision.
Contact Information : Ms. McKenzie Boschitsch
mckmb@nursing.upe

JOB_ID : #22785
Title : Tax Associate Intern
Posted On : 09/21/20
Summary : Every year, the Campaign for Working Families, Inc. recruits, trains, and manages hundreds of volunteers and interns who staff free tax sites across Philadelphia and surrounding counties, as well as South Jersey.  In 2018, CWF's Philadelphia and South Jersey tax sites provided over 33,000 taxpayers with access to important tax credits through free tax preparation, resource building and asset development services. Since our founding, we have brought in over $398 million in refunds to our regionï¿½s economy.

For this upcoming tax season, CWF is seeking highly-motivated and mature individuals with above-average PC skills to fill our Tax Associate intern positions.  The position is ideal for students with WorkStudy grants; however, all students are encouraged to apply as each will learn critical skills that will enhance your resume and prospects for employment upon graduation.  

Individuals should possess strong 

JOB_ID : #23761
Title : Program Assistant
Posted On : 09/01/20
Summary : The Program Assistant will provide administrative support to the Critical Writing Program. Effective communication and coordination with other assistants is essential. This position provides excellent hands-on experience for students wishing to enhance their computer, administration, and public relations skills within a large, multi-faceted program.*Excellent organizational, communicational, and interpersonal skills;
*Ability to work independently as well as collaboratively, with limited supervision in both cases;
*Willingness to take initiative;
*Proficiency with Macintosh and Windows operating systems;
*Intermediate knowledge of modern browsers, such as Google Chrome or Firefox;
*Competence with Microsoft Excel filers, formulae, and formatting.Please send a resume, cover letter, and class schedule to critwrit@writing.upenn.edu
Details : Work-Study Only1 position available for academic year 
$13.00 per hour
 / 10

JOB_ID : #19943
Title : Student Linguistics & Communication RA
Posted On : 01/19/21
Summary : The Children's Hospital of Philadelphia Research Institute is an interdisciplinary institution dedicated to conducting basic, clinical and translational research on conditions and diseases that affect children, and is part of The Children's Hospital of Philadelphia and University of Pennsylvania research community.

The Center for Autism Research (CAR) is a collaborative effort between The Children's Hospital of Philadelphia and the University of Pennsylvania. CAR's goals are to identify the causes of autism spectrum disorders (ASD), develop effective treatments, and to improve quality of life for individuals with ASD and their families. 
The Student Linguistics and Communication Research Assistant will work with Infant Brain Imaging Study (IBIS) speech and language team at CAR on a variety of tasks that support the research aims. The IBIS study is a federally funded, multi-site, longitudinal 

...

JOB_ID : #25017
Title : Group Study Facilitator
Posted On : 10/06/20
Summary : Group Study facilitators will lead an effective study group learning session of ideally 3-4 members (fellow students) in designated coursework study.  The facilitator is expected to make a serious commitment to keeping the group on track with weekly agenda assignments, coursework and concept review, and the goal of creating a collaborative study environment, and ultimately a noted skill development and knowledge acquisition.Applicants should have basic proficiency in course material
Applicants must commit to attending pre-program training and complete approximately ten (10) hours of orientation and training.
Applicants are expected to participate in continuing education and/or check-in meetings with Wharton Undergraduate Division staff members periodically throughout the semester
Details : Work-Study Only3 positions available for academic year 
$15.00 per hour
 / 6 hours per week
 
Requires highly developed 

JOB_ID : #24921
Title : Student Research Assistant
Posted On : 09/11/20
Summary : The Student Research Assistant will be housed at CHOP's Gender and Sexuality Development Clinic (GSD Clinic) and at PolicyLab. The GSD Clinic offers psychosocial and medical support for gender variant, gender expansive and transgender youth up to age 21 and their families. The multidisciplinary team, led by Nadia Dowshen, MD, MSHP, and Linda Hawkins, PhD, MSEd, LPC, includes specialists in gender identity development from Social Work and Family Services, Adolescent Medicine, Endocrinology, and Behavioral Health. 

In this role, which requires a minimum of a one-year commitment, the student research volunteer will work mainly on the GSD Clinic Registry, but will also be given various research duties as appropriate for other projects. The volunteer will work under the direction of the Research Coordinator as well as the Co-Directors of the GSD Clinic. 
Responsibilities of the volunteer include: 
- Conductin

JOB_ID : #22971
Title : Nanny
Posted On : 10/30/20
Summary : Loving family in University City looking for an energetic, caring and responsible nanny with a mild temperament to help care for our 5-year-old twin girls. 

Candidates should be willing to play, read, help with arts & crafts, and take them on walks to the park, library, etc. Some light tidying up would be part of our caregiver's responsibilities. Please be comfortable with pets (we have a cat). Duties will include pickups and drop-offs at school when in person school resumes. Seeking someone to handle meal preparation and the kids' laundry. After school duties include homework help. Looking for someone to start in December.

Approximately 35 hours/week. Wage is negotiable. We are interviewing now.Prior child care/nanny experience with references.
High energy, creative and loves children.
Comfortable asking questions.
First aid, CPR and other Child Care Certifications preferred.
Driver's License preferred.Please contact Peter

JOB_ID : #25217
Title : Inside Sales Partner
Posted On : 01/27/21
Summary : Mersi Distribution is a fast-growing distributor of safety products and workwear. We offer private label manufacturing services as well (most recent project involved manufacturing cloth masks for one of the country's largest fast food chains). We are seeking an Inside Sales Partner to join the team and accelerate our sales efforts. If you love sales, this opportunity is for you.

- Lead the creation and implementation of outbound sales strategy and system
- Create outbound email and call templates
- Send outbound emails and/or make calls to leads on a daily basis
- Optimize sales tools so we're targeting the right leads and following up with leads methodically, periodically, and appropriately (lead gen, CRM, etc.)
- Address prospective customer inquiries and questions promptly with zealous communication (our policy is to respond <15min, even if it's just a note that you're fetching info for them)
- Manage accou

JOB_ID : #25001
Title : Tutor
Posted On : 09/28/20
Summary : Conversational English Tutors assist foreign students in small groups to develop their English skills through practice via zoom.Education majors are preferred or individuals with tutoring experience. No foreign language experience required.
Details : Non Work-Study, Non Penn Affiliated10 positions available for academic year 
$15.00 per hour
 / 20 hours per week
 
Requires highly developed skills

Highly developed skills, completion of related upper-level course work or graduate-level course work may be required. Responsibilities can include regular independent supervision and/or coordination of programs and/or projects involving highly complex equipment, analysis of data. Often requires the ability to communicate skills or expertise to others. Regular independent decision-making. The decisions of students in these positions could directly affect the operation and success of the project or program. Limited, if any, supervisio

## Condense information into Dataframe

In [21]:
df=pd.DataFrame(jobs_list)
for i in range(0,df.shape[0]):
    df.WeeklyPay[i]=float(str(df.WeeklyPay[i]).replace("$",""))

### Sorting by Weekly Pay (Descending Order)

In [22]:
df.sort_values(by=['WeeklyPay'],ascending=False)

,JOB_ID,Title,Posted On,Summary,Details,Contact Information,Link,PayPerHour,HoursPerWeek,WeeklyPay
212,#24791,Full-time elementary tutor/teacher,08/19/20,Our family is looking for an experienced tutor...,"Non Work-Study, Non Penn Affiliated1 position ...",Mr. Jeffrey Marrazzo\njeffrey.marrazzo@gmail.com,https://www.sfs.upenn.edu/seo/job_search/job_s...,$15.00,20,300
116,#24432,Creative Content Writer,10/15/20,Penn's Online Learning Initiative is looking f...,Both Work-Study and Non Work-Study1 position a...,Ms. Anna Miller\najmiller@upenn.edu\nVan Pelt ...,https://www.sfs.upenn.edu/seo/job_search/job_s...,$15.00,20,300
65,#24798,Graduate Assistant,08/20/20,This position reports to the Director of GSE C...,Work-Study Only1 position available for academ...,Dr. Amy Benedict\namyben@upenn.edu\n3700 Walnu...,https://www.sfs.upenn.edu/seo/job_search/job_s...,$15.00,20,300
217,#25071,Educator/Nanny/Sitter Position,10/27/20,"Hello, My name is Jessica Daly. I graduated fr...","Non Work-Study, Non Penn Affiliated1 position ...",Ms. Jessica Daly\njessica@jessicadalyjd.com,https://www.sfs.upenn.edu/seo/job_search/job_s...,$15.00,20,300
215,#24771,Lower Merion child care this fall,08/15/20,Calling students who live in/near Lower Merion...,"Non Work-Study, Non Penn Affiliated1 position ...",Dr. Evan Fieldston\nevanfieldston@gmail.com,https://www.sfs.upenn.edu/seo/job_search/job_s...,$15.00,20,300
...,...,...,...,...,...,...,...,...,...,...
56,#14849,Fisher Hassenfeld Graphic Designer,09/25/20,Design creative and unique weekly FH Chronicle...,Both Work-Study and Non Work-Study2 positions ...,Ms. Felicia Goodson\nfgoodson@upenn.edu\n3700 ...,https://www.sfs.upenn.edu/seo/job_search/job_s...,$8.75,5,43.75
219,#24871,Tutor needed,08/29/20,Looking for tutor to help rising 7th grader wi...,"Non Work-Study, Non Penn Affiliated1 position ...",Dr. Emily Hage\nemilykhage@gmail.com,https://www.sfs.upenn.edu/seo/job_search/job_s...,$10.00,4,40
159,#24773,Research Assistant for a Clinical Study,08/16/20,This job involves providing assistance on a cl...,Work-Study Only2 positions available for acade...,Dr. Edward Brodkin\nebrodkin@pennmedicine.upen...,https://www.sfs.upenn.edu/seo/job_search/job_s...,$10.00,4,40
208,#15224,Grader 430,09/02/20,Grader for STAT 430. Requires excellent famil...,Work-Study Only2 positions available for acade...,Ms. Carol M. Reich\nreichc@wharton.upenn.edu\n...,https://www.sfs.upenn.edu/seo/job_search/job_s...,$13.00,3,39


### Saving Information to Excel File

In [23]:
df.to_excel("PennJobs.xlsx")

In [25]:
df.head()

,JOB_ID,Title,Posted On,Summary,Details,Contact Information,Link,PayPerHour,HoursPerWeek,WeeklyPay
0,#25176,RESEARCH ASSISTANT,01/08/21,The Penn Collaborative for CBT and Implementat...,Work-Study Only1 position available for academ...,Mrs. Eileen Mergliano\nemerg@pennmedicine.upen...,https://www.sfs.upenn.edu/seo/job_search/job_s...,$10.71,10,107.1
1,#24951,IT Support Specialist,09/11/20,This is a *remote work* position. Provide IT ...,Work-Study Only2 positions available for acade...,Mr. Richard Cardona\nrcardona@asc.upenn.edu\n3...,https://www.sfs.upenn.edu/seo/job_search/job_s...,$15.00,20,300
2,#25171,BAM Lab Research Assistant - Spanish,01/06/21,The primary purpose of this position is to hel...,Work-Study Only1 position available for academ...,Dr. Morgan Hoke\nmhoke@sas.upenn.edu\n3260 Sou...,https://www.sfs.upenn.edu/seo/job_search/job_s...,$12.50,15,187.5
3,#25224,Equipment Aide,01/21/21,This position ensures the availability of unif...,Work-Study Only1 position available for academ...,Mr. Joshua LaRosa\njlarosa@upenn.edu\n235 S. 3...,https://www.sfs.upenn.edu/seo/job_search/job_s...,$8.40,20,168
4,#25201,Operations Assistant,01/14/21,The operations Assistant will support DRIA wit...,Work-Study Only10 positions available for acad...,Mr. Michael Mantz\nmantz@upenn.edu\n235 S. 33r...,https://www.sfs.upenn.edu/seo/job_search/job_s...,$8.00,20,160


# Contact Information:
(For Collaboration on Projects or building web automation bots)

## https://www.fiverr.com/rick137codes